In [1]:
import pandas as pd
from os import path
from Bio import SeqIO
import gzip
from tqdm import tqdm
from glob import glob
import os
import pymysql
import shutil 
import urllib.request
import sys
import random
import numpy as np
sys.path.append("../")
from db_mapping import save_mapping_to_db
con = pymysql.connect(host='127.0.0.1', unix_socket='/opt/lampp/var/mysql/mysql.sock', user='root', passwd='', db='current_project')
cur = con.cursor()

In [2]:
def fetch_transcript(transcript_id):
    if 'ENST' in transcript_id: 
        transcript_id_no_vs = transcript_id.split('.')[0]
        url = 'https://rest.ensembl.org/sequence/id/'+transcript_id_no_vs+'?content-type=text/x-fasta;type=protein'
    else:
        url = 'https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=protein&report=fasta&id='\
                        +transcript_id+'&extrafeat=null&conwithfeat=on'

    fp = urllib.request.urlopen(url)
    mybytes = fp.read()

    sequence = mybytes.decode("utf8")
    fp.close()
    sequence = '\n'.join(sequence.split('\n')[1:])
    sequence = '>'+transcript_id+' [Homo sapiens]\n' + sequence

    return sequence


In [3]:
transcript_lists = {}
fasta_lists = {}


In [ ]:
df_ensts = pd.concat([pd.read_sql("SELECT DISTINCT canonical_transcript FROM gnomad", con).iloc[:,0],
          pd.read_sql("SELECT DISTINCT accession_number FROM CosmicMutantExport WHERE accession_number "+\
                      "LIKE 'ENST%'", con).iloc[:,0]])
df_ensts = df_ensts.unique()
pd.Series(df_ensts).to_csv('/opt/current_project/db/mapping/WHOLE_ENST_IDS.csv', header=None, index=None)

In [ ]:
project_path = '/opt/current_project' 
db_path = '/opt/current_project/db'
sequences_path = path.join(project_path, 'results', 'sequences')

if path.exists(sequences_path):
    shutil.rmtree(sequences_path)

os.makedirs(sequences_path)

all_relevant_fasta_sequences = ''


# Fetch the transcript ids that have variants
transcripts_in_the_db = pd.concat([pd.read_sql("SELECT DISTINCT nm_id FROM clinvar", con).iloc[:,0], 
          pd.read_sql("SELECT DISTINCT canonical_transcript FROM gnomad", con).iloc[:,0],
          pd.read_sql("SELECT DISTINCT accession_number FROM CosmicMutantExport WHERE accession_number "+\
                      "LIKE 'ENST%'", con).iloc[:,0]],
          axis=0)
transcripts_in_the_db = transcripts_in_the_db.unique()

nm_to_transcript_id = pd.read_csv(path.join(db_path, 'mapping', 'NM_NP_GeneID.list'), sep='\t', header=None).iloc[:, :2]
nm_to_transcript_id.columns = ['retrieval_id', 'transcript_id']
enst_to_transcript_id = pd.read_csv(path.join(db_path, 'mapping', 'ENST_ENSTV_GeneID.list')).iloc[:, :2]
enst_to_transcript_id.columns = ['retrieval_id', 'transcript_id']


retrieval_id_to_transcript_id = pd.concat([nm_to_transcript_id, enst_to_transcript_id], axis=0, ignore_index=True)
retrieval_id_to_transcript_id = retrieval_id_to_transcript_id.loc[
                                retrieval_id_to_transcript_id['retrieval_id'].drop_duplicates().index, :]

retrieval_id_to_transcript_id = retrieval_id_to_transcript_id.set_index('retrieval_id')

retrieval_id_to_transcript_id = retrieval_id_to_transcript_id.loc[retrieval_id_to_transcript_id.index.\
                                                                  intersection(transcripts_in_the_db)]
retrieval_id_to_transcript_id = retrieval_id_to_transcript_id.set_index('transcript_id')

species_list = ['Homo_sapiens', 'Homo_sapiens_Ensembl']

if not path.exists(sequences_path):
    os.makedirs(sequences_path)
    
fasta_lists['Homo_sapiens'] = {}

for species in tqdm(species_list):
        
    fasta_sequence_file = path.join(db_path, 'proteins', species+'.faa')

    fasta_sequences = SeqIO.parse(fasta_sequence_file, "fasta")
    for fasta in tqdm(fasta_sequences):
        transcript_id, sequence = fasta.id, str(fasta.seq)
        if species == 'Homo_sapiens_Ensembl':
            description_map = {desc.split(':')[0]: desc.split(':')[1] for desc in fasta.description.split(' ') if 
                                  len(desc.split(':')) > 1}
            transcript_id = description_map['transcript']
        sequence = '>'+transcript_id+' [Homo sapiens]\n' + sequence
        
        filename = path.join(sequences_path, transcript_id+'.fasta')

        if transcript_id not in retrieval_id_to_transcript_id.index:
            continue

        with open(filename, 'w') as f:
            f.write(sequence)

        fasta_lists['Homo_sapiens'][transcript_id] = sequence


In [ ]:
for transcript_id in tqdm(retrieval_id_to_transcript_id.index.difference(fasta_lists['Homo_sapiens'])):
    try:
        fasta_lists['Homo_sapiens'][transcript_id] = fetch_transcript(transcript_id)    
        print(fasta_lists['Homo_sapiens'][transcript_id])
    except Exception as e:
        print(e, transcript_id)

In [ ]:

human_transcripts = glob(path.join(sequences_path, '*'))
homology_list = pd.read_csv(path.join(db_path,'orthology','Homology.list'), sep='\t', dtype=str)

species_list = ['Pan_troglodytes', 'Macaca_mulatta', 'Rattus_norvegicus', 'Mus_musculus', 'Danio_rerio', 
            'Xenopus_tropicalis', 'Drosophila_melanogaster', 'Caenorhabditis_elegans']
homology_list.columns = ['Homo_sapiens'] + species_list
homology_list = homology_list.set_index('Homo_sapiens')

enst_to_gene_id = pd.read_csv(path.join(db_path, 'mapping', 'NewCurated_ENSTvsGENEID.csv'))
enst_to_gene_id.columns = ['transcript_id', 'gene_id']
np_to_gene_id = pd.read_csv(path.join(db_path, 'mapping', 'NM_NP_GeneID.list'), sep='\t').iloc[:, 1:] 
np_to_gene_id.columns = ['transcript_id', 'gene_id']

transcript_id_to_gene_id = pd.concat([enst_to_gene_id, np_to_gene_id], axis=0, ignore_index=True)

gene_id_to_transcript_ids = pd.concat([enst_to_gene_id, np_to_gene_id], axis=0, ignore_index=True)

for species in species_list:
    species_list_file = path.join(db_path, 'mapping', 'NP_tables', species+'.list')
    print(species_list_file)
    transcript_lists[species] = pd.read_csv(species_list_file, sep='\t')
    transcript_list = transcript_lists[species][['Protein product', 'GeneID']]
    transcript_list.columns = ['transcript_id', 'gene_id']
    
    transcript_id_to_gene_id = pd.concat([transcript_id_to_gene_id, transcript_list],
                                        axis=0, ignore_index=True)
    gene_id_to_transcript_ids = pd.concat([gene_id_to_transcript_ids, transcript_list],
                                        axis=0, ignore_index=True)
    
    fasta_lists[species] = {} 
    fasta_sequence_file = path.join(db_path, 'proteins', species+'.faa')
    
    fasta_sequences = SeqIO.parse(fasta_sequence_file, "fasta")
    for fasta in tqdm(fasta_sequences):
        transcript_id, sequence = fasta.id, str(fasta.seq)
        transcript_id = transcript_id.split('.')[0]
        fasta_lists[species][transcript_id] = sequence
    
    

transcript_id_to_gene_id = transcript_id_to_gene_id.dropna()
transcript_id_to_gene_id.loc[:, 'transcript_id'] = transcript_id_to_gene_id['transcript_id'].apply(lambda x: x.split('.')[0])
transcript_id_to_gene_id.loc[:, 'gene_id'] = transcript_id_to_gene_id.loc[:, 'gene_id'].astype(str) 
transcript_id_to_gene_id = transcript_id_to_gene_id.set_index('transcript_id')

gene_id_to_transcript_ids = gene_id_to_transcript_ids.dropna()
gene_id_to_transcript_ids.loc[:, 'gene_id'] = gene_id_to_transcript_ids.loc[:, 'gene_id'].astype(str)

gene_id_to_transcript_ids.loc[:, 'transcript_id'] = gene_id_to_transcript_ids['transcript_id'].apply(lambda x: x.split('.')[0])
gene_id_to_transcript_ids = gene_id_to_transcript_ids.groupby('gene_id')['transcript_id'].apply(lambda x: ','.join(x))


In [ ]:

seqs_with_homology_path = '/opt/current_project/results/seqs_with_homology/'

if path.exists(seqs_with_homology_path):
    shutil.rmtree(seqs_with_homology_path)

os.makedirs(seqs_with_homology_path)

for transcript_file in tqdm(human_transcripts):
    with open(transcript_file, 'r') as f:
        current_fasta = f.read()
    human_transcript_id_orig = current_fasta.split('\n')[0].split(' ')[0].replace('>', '') 
    human_transcript_id = human_transcript_id_orig.split('.')[0]
    if human_transcript_id not in transcript_id_to_gene_id.index:
        continue
        
    if pd.isna(transcript_id_to_gene_id.loc[human_transcript_id, 'gene_id']):
        continue
    if ' ' in str(transcript_id_to_gene_id.loc[human_transcript_id, 'gene_id']):
        print(transcript_id, 
              transcript_id_to_gene_id.loc[human_transcript_id, 'gene_id'])
        continue
    
    human_gene_id = transcript_id_to_gene_id.loc[human_transcript_id, 'gene_id']
    
    if human_gene_id not in homology_list.index:
        continue
    
    transcript_ids = set()
    
    for species, homolog_gene_ids in homology_list.loc[human_gene_id].to_dict().items():
        if pd.isna(homolog_gene_ids):
            continue
        homolog_gene_ids = homolog_gene_ids.split(',')
        for gene_id in homolog_gene_ids:
            if gene_id == '':
                continue
            if gene_id not in gene_id_to_transcript_ids.index:
                continue
            
            homolog_transcript_ids = gene_id_to_transcript_ids.loc[gene_id].split(',')
            
            for transcript_id in homolog_transcript_ids:
                if transcript_id not in fasta_lists[species]:
                    continue
                if transcript_id in transcript_ids:
                    continue
                
                if len(homolog_transcript_ids) > 3 and random.randint(1,2) == 1:
                    continue

                transcript_ids.add(transcript_id)
                
                current_fasta += '\n'
                current_fasta += '>'+transcript_id+' ['+species.replace('_',' ')+']'
                current_fasta += '\n'+fasta_lists[species][transcript_id]
    
    with open(path.join(seqs_with_homology_path, human_transcript_id_orig +'.fasta'), 'w') as f:
        f.write(current_fasta)


In [9]:

blast_db_path = path.join(db_path, 'blast_db/')

if path.exists(blast_db_path):
    shutil.rmtree(blast_db_path)

os.makedirs(blast_db_path)

all_fasta = '\n'.join(list(fasta_lists['Homo_sapiens'].values()))

curated_file_path = path.join(blast_db_path, 'convart_curated_fasta.fasta')

with open(curated_file_path, 'w') as f:
    f.write(all_fasta)


os.system("makeblastdb -in "+curated_file_path+" -out "+\
    path.join(blast_db_path, 'convartProteome')+\
    " -title "+path.join(blast_db_path, 'convartProteome')+" -dbtype prot")



0

In [10]:
print("makeblastdb -in "+curated_file_path+" -out "+\
    path.join(blast_db_path, 'convartProteome')+\
    " -title "+path.join(blast_db_path, 'convartProteome')+" -dbtype prot")

makeblastdb -in /opt/current_project/db/blast_db/convart_curated_fasta.fasta -out /opt/current_project/db/blast_db/convartProteome -title /opt/current_project/db/blast_db/convartProteome -dbtype prot


In [ ]:
!python create_alignments.py construct_database

In [63]:
df_matching = pd.read_sql("""SELECT GROUP_CONCAT(if(id like 'NP%', id, null)) AS np_id,
    GROUP_CONCAT(if(id like 'ENST%', id, null)) AS enst_id 
    FROM `convart_gene` WHERE species_id = 'Homo sapiens' GROUP BY sequence HAVING COUNT(id) > 1 AND np_id is NOT NULL 
    ORDER BY id ASC
    """, con)
df_matching.loc[:, 'enst_id'] = res.loc[:, 'enst_id'].replace({None: ""})

df_matching = explode_str(df_matching, 'enst_id', ',')
df_matching = explode_str(df_matching, 'np_id', ',')

df_matching = df_matching.set_index('np_id')
df_matching.columns = ['ENST']

save_mapping_to_db(df_matching, con, cur)


In [67]:
df_np_ncbi = pd.read_sql("""SELECT nc_hum.meta_value AS np_id, nc_hum.ncbi_gene_id FROM ncbi_gene_meta AS nc_hum
        WHERE meta_key='protein_number' AND meta_value LIKE 'NP%' GROUP BY nc_hum.meta_value""", con)
df_np_ncbi = df_np_ncbi.set_index('np_id')

In [94]:
df_matching_with_ncbi = df_matching.copy()
df_matching_with_ncbi.loc[:, 'ncbi_id'] = df_matching_with_ncbi.index.map(lambda x: df_np_ncbi.loc[x, 'ncbi_gene_id'])
for _, row in df_matching_with_ncbi.iterrows():
    cur.execute("INSERT IGNORE INTO ncbi_gene_meta (ncbi_gene_id, meta_key, meta_value) VALUES (%s, 'protein_number', %s)", 
                        (row['ncbi_id'], row['ENST']))